# Using the Neo4j Graph Data Science Library from Python

In [1]:
from neo4j import GraphDatabase

In [2]:
# TODO: set your own connection credentials here
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "<YOUR_PASSWORD>"))

## Creating the named projected graph

In [3]:
graphName = "my_complex_projected_graph"

nodeProj = {
    "User": {
        "label": "User",
    }
}

relProj = {
    "FOLLOWS": {
        "type": "FOLLOWS",
        "orientation": "UNDIRECTED",
        "aggregation": "SINGLE"
    }
}

In [4]:
cypher = "CALL gds.graph.create( $graphName, $nodeProj, $relProj)"

In [5]:
with driver.session() as session:
    result = session.run(
        cypher,
        graphName=graphName,
        nodeProj=nodeProj,
        relProj=relProj
    )
    display(result.data())

[{'graphName': 'my_complex_projected_graph',
  'nodeProjection': {'User': {'properties': {}, 'label': 'User'}},
  'relationshipProjection': {'FOLLOWS': {'orientation': 'UNDIRECTED',
    'aggregation': 'SINGLE',
    'type': 'FOLLOWS',
    'properties': {}}},
  'nodeCount': 596,
  'relationshipCount': 1192,
  'createMillis': 69}]

**WARNING** if you run the above cell twice, you will get the following error message:

> ClientError: Failed to invoke procedure `gds.graph.create`: Caused by: java.lang.IllegalArgumentException: A graph with name 'my_complex_projected_graph' already exists.

No worry, it just means the GDS can't recreate an existing projected graph. You can run the rest of this notebook safely. If you want to, the queries to drop this projected graph is at the end of this notebook.


## Running an algorithm: example with PageRank

### Algorithm configuration dict

In [6]:
algoConfig = {
    "dampingFactor": 0.85,
    # "relationshipWeightProperty": "weight"
}

In [7]:
with driver.session() as session:
    result = session.run(
        "CALL gds.pageRank.stream($graphName, $algoConfig)",
        graphName=graphName,
        algoConfig=algoConfig,
    )
    for record in result:
        display(record.data())

{'nodeId': 0, 'score': 0.15000000000000002}

{'nodeId': 1, 'score': 1.2104374413844197}

{'nodeId': 2, 'score': 0.9438581566326321}

{'nodeId': 3, 'score': 0.4038160189986229}

{'nodeId': 4, 'score': 0.6520907054888087}

...

{'nodeId': 591, 'score': 0.15000000000000002}

{'nodeId': 592, 'score': 0.15000000000000002}

{'nodeId': 593, 'score': 0.15000000000000002}

{'nodeId': 594, 'score': 0.15000000000000002}

{'nodeId': 595, 'score': 0.15000000000000002}

### Using `gds.util.asNode` to get more meaningful results

In [8]:
with driver.session() as session:
    result = session.run(
        """CALL gds.pageRank.stream($graphName, $algoConfig)
            YIELD nodeId, score
            RETURN gds.util.asNode(nodeId) as node, score
        """,
        graphName=graphName,
        algoConfig=algoConfig,
    )
    for record in result:
        print(record.get("node").get("user_id"), record.get("score"))

1 0.15000000000000002
31 1.2104374413844197
32 0.9438581566326321
33 0.4038160189986229
34 0.6520907054888087
35 1.733900258783251
36 1.970460276398808
37 0.4038160189986229
38 2.8126600407063966
39 0.9132519175764172
40 9.010058941878375
41 11.484363865479828
42 10.334434717893597
43 3.51371067268774
44 4.429150780197232
45 12.88761965315789
46 10.417098037712279
47 1.8435446158517153
48 14.709171251580118
49 2.159772182954475
50 9.270769257098436
51 5.6819858958479035
52 10.651387471333145
53 4.655745753366499
54 3.9165744459256526
55 10.488682551681997
56 7.839183766581118
57 7.951555732078848
58 1.4411550694378095
59 9.722989419475192
60 13.774939332786015
61 10.313357470557094
62 2.1540201521478597
63 10.885534165613354
64 1.0494503078283743
65 5.135402509756387
66 1.3217056492343546
67 11.624030757509173
68 11.090736489370467
69 1.0808968721888963
80 0.4038160189986229
81 0.4038160189986229
82 0.4038160189986229
83 0.4038160189986229
84 0.664977219211869
85 0.4038160189986229
86 

13807 0.15000000000000002
13808 0.15000000000000002
13824 0.15000000000000002
13848 0.15000000000000002
13879 0.15000000000000002
13886 0.15000000000000002
13914 0.15000000000000002
13938 0.15000000000000002
13940 0.15000000000000002
13968 0.15000000000000002
13991 0.15000000000000002
14017 0.15000000000000002
14027 0.15000000000000002
14034 0.15000000000000002
14057 0.15000000000000002
14079 0.15000000000000002
14110 0.15000000000000002
14113 0.15000000000000002
14143 0.15000000000000002
14162 0.15000000000000002
14163 0.15000000000000002
14192 0.15000000000000002
14204 0.15000000000000002
14217 0.15000000000000002
14242 0.15000000000000002
14272 0.15000000000000002
14300 0.15000000000000002
14318 0.15000000000000002
14344 0.15000000000000002
14371 0.15000000000000002
14375 0.15000000000000002
14401 0.15000000000000002
14428 0.15000000000000002
14455 0.15000000000000002
14483 0.15000000000000002
14510 0.15000000000000002
14541 0.15000000000000002
14569 0.15000000000000002
14598 0.1500

### Writing results back to the graph

In [9]:
algoConfig["writeProperty"] = "pr"

with driver.session() as session:
    result = session.run(
        "CALL gds.pageRank.write($graphName, $algoConfig)",
        graphName=graphName,
        algoConfig=algoConfig,
    )
    display(result.single().data())

{'nodePropertiesWritten': 596,
 'createMillis': 0,
 'computeMillis': 484,
 'writeMillis': 37,
 'ranIterations': 20,
 'didConverge': False,
 'configuration': {'maxIterations': 20,
  'writeConcurrency': 4,
  'sourceNodes': [],
  'writeProperty': 'pr',
  'relationshipWeightProperty': None,
  'nodeLabels': ['*'],
  'sudo': False,
  'dampingFactor': 0.85,
  'relationshipTypes': ['*'],
  'cacheWeights': False,
  'tolerance': 1e-07,
  'concurrency': 4}}

## Dropping a named projected graph

In [10]:
with driver.session() as session:
    result = session.run(
        "CALL gds.graph.drop($graphName)",
        graphName=graphName,
    )
    display(result.data())

[{'graphName': 'my_complex_projected_graph',
  'nodeProjection': {'User': {'properties': {}, 'label': 'User'}},
  'relationshipProjection': {'FOLLOWS': {'orientation': 'UNDIRECTED',
    'aggregation': 'SINGLE',
    'type': 'FOLLOWS',
    'properties': {}}},
  'nodeQuery': None,
  'relationshipQuery': None,
  'nodeCount': 596,
  'relationshipCount': 1192,
  'degreeDistribution': {'p99': 30,
   'min': 0,
   'max': 48,
   'mean': 2.0,
   'p90': 2,
   'p50': 1,
   'p999': 39,
   'p95': 6,
   'p75': 1},
  'creationTime': neo4j.time.DateTime(2020, 8, 12, 11, 54, 59.462),
  'modificationTime': neo4j.time.DateTime(2020, 8, 12, 11, 54, 59.552)}]